# Training a Classifier

## Imports and setup

First we'll set some options up to make everything display correctly. It's mostly because these assault descriptions can be quite long, and the default is to truncate text after a few words.

In [1]:
import pandas as pd

pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 300)

%matplotlib inline

/Users/misha/.pyenv/versions/3.10.0/lib/python3.10/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## Read in our data

Our dataset is going to be a database of crimes committed between 2008 and 2012. The data has been cleaned and filtered a bit, though, so we're only left with two columns:

* `CCDESC`, what criminal code was violated
* `DO_NARRATIVE`, a short text description of what happened

We're going to use this description to see if we can separate serious cases of assault compared to non-serious cases of assault.

In [2]:
X_train = pd.read_csv('assaults_downgraded_train.csv')
df = X_train
df

,Unnamed: 0,CCDESC,DO_NARRATIVE,serious,downgraded
0,823014,BATTERY - SIMPLE ASSAULT,DO-SUSP AND VICT INVOLVED IN ARGUMENT SUSP GRABBED VICT BY NECK PUSHED HERAGAINST WALL AND CHOKED HER,0,0
1,660053,CHILD ABUSE (PHYSICAL) - SIMPLE ASSAULT,DO-V1 AND V2 WERE DISCIPLINED BY STEPFATHER FRIDAY AFTER SCHOOL WITH A BELT V1 HAD SOME MINOR BRUSING ON BODY V2 HAD BRUISING ON HIS BODY,0,0
2,736013,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",DO-UNK SUSPS DROVE UP ALONSIDE VICTS S YELLED FUCK 38 S1 THEN SHOT APPROX 4-5 RONDS FROM BLUE STEEL REVOLVER STRIKING V1 V2,1,0
3,255680,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",DO-SUSP AND VICT BECAME INV IN A DISPUTE THAT ESCALATED WHEN THE SUSP HIT VICT ON HER LEFT ARM WITH A BAT,1,0
4,796623,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",DO-VICT WALKED W B VENTRUA BLVD SUSPS DROVE PAST AND FIRED MULTIPLE BBS ATVICT VICT WERE STRUCK ONCE,1,0
...,...,...,...,...,...
165860,26664,BATTERY - SIMPLE ASSAULT,DO-SUSP AND VICT WERE INVOLVED IN A VERBAL ALTERCATION SUSP BECAME ANGRY STRUCK THE VICT LEAVING VISIBLE INJURY SUSP THEN FLED THE LOCATION,0,0
165861,450551,INTIMATE PARTNER - SIMPLE ASSAULT,DO-VICT AND SUSP BECAME INV IN VERBAL ARGUMENT SUSP BECAME ANGRY AND GRAB VICT HAIR THROWING VICT TO THE FLOOR,0,0
165862,758754,INTIMATE PARTNER - SIMPLE ASSAULT,DO-ON ABOVE DATE AND TIME S BECAME ANGRY AT V OVER AN ARGUEMENT S STRUCK VIN FACE WITH CLOSED FIST CAUSING BRUISES S FLED LOCATION,0,0
165863,273702,INTIMATE PARTNER - SIMPLE ASSAULT,DO-SUSP AND VICT ENGAGED IN VERBAL DISPUTE SUSP BECAME ANGRY STRUCK VICT MULTIPLE TIMES TO HER FACE AND ARMS CAUSING BRUISING VICT FLED LOC W CHILD,0,0


# Time to classify

Now we're going to build a classifier that can predict whether an assault is serious or non-serious. To do that, we first need to count all the words in each description.

## Stemming

When we talk about "words," we can mean a few different things. Do you feel like "stab," "stabbed," and "stabbing" should all count the same? If yes, you're interested in **stemming.**

Stemming is the process of trying to convert words to their root. There are a few different kinds of stemmers, but we'll be using one called **SnowballStemmer** that comes with NLTK.

In [3]:
from nltk.stem import SnowballStemmer

stemmer = SnowballStemmer('english')

Let's see how it works in the case of different tenses of the word "stabbing:"

In [4]:
from nltk.stem import SnowballStemmer

stemmer = SnowballStemmer('english')

print(stemmer.stem("stab"))
print(stemmer.stem("stabbed"))
print(stemmer.stem("stabbing"))

stab
stab
stab


Looks good! It also works with singular and plurals.

In [5]:
print(stemmer.stem("gun"))
print(stemmer.stem("guns"))

gun
gun


It can have trouble with more complicated words, though. Here's an example with "knife" as compared to "knives."

In [6]:
import nltk
# nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /Users/misha/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


True

In [7]:
print(stemmer.stem("knife"))
print(stemmer.stem("knifed"))
print(stemmer.stem("knives"))

# from nltk.stem import WordNetLemmatizer 
# lemmatizer = WordNetLemmatizer()
# print(lemmatizer.lemmatize('knife'))
# print(lemmatizer.lemmatize('knifed'))
# print(lemmatizer.lemmatize('knife'))

knife
knife
knive


If it gets most of what we're looking for, though, I think we're okay.

## Building a vectorizer that uses stemming

Vectorizing is the process of converting words to numbers, and counting each of them (along with any other transformations). It's a little complex to incorporate stemming into the process, but here we go.

We're also using a `TfidfVectorizer`, which doesn't count words exactly. More common words become less meaningful, and words in short sentences mean more than words in longer sentences.

For example, your name showing up in a tweet probably means the tweet is about you. If your name shows up once in a big, big book, though, the chances are much lower that the entire book is about you.

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

stemmer = SnowballStemmer('english')
class StemmedTfidfVectorizer(TfidfVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedTfidfVectorizer,self).build_analyzer()
        return lambda doc:(stemmer.stem(word) for word in analyzer(doc))

Now we'll actually use our vectorizer to count the words. We're being a little picky, in that:

* We're only using words that show up at least 15 times in our dataset, and
* We're ignoring anything that shows up in more than half of the police reports

This helps cut down on noise. Adjusting the terms will give us a greater or fewer number of words, and might also affect our accuracy later on.

## Feature Engineering

Check out the documentation for TfIdfVectorizer and CountVectorizer
- https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
- https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

You can tweak the parameters in the code below and see how this vectorizer performs. For example, what happens when you modify `min_df` and `max_df`? What about adding some things to the `stop_words` list? Setting `binary` to True instead of False? 

Let's start with the default and you can come back to modify some things later!

In [10]:
%%time

# Check out the documentation for TfIdfVectorizer and CountVectorizer
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
vectorizer = StemmedTfidfVectorizer(min_df=15, max_df=0.25, binary=True)

X = vectorizer.fit_transform(df.DO_NARRATIVE)
X

CPU times: user 1min 29s, sys: 7.48 s, total: 1min 37s
Wall time: 2min 24s


<165865x3416 sparse matrix of type '<class 'numpy.float64'>'
	with 2118534 stored elements in Compressed Sparse Row format>

If we're only paying attention to words that show up at least 15 times, and are ignoring words that show up in half of the reports, how many words we we using?

In [11]:
len(vectorizer.get_feature_names())

/Users/misha/.pyenv/versions/3.10.0/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


3416

While it doesn't do us much good in terms of analysis, we can take a pick look at our words and sentences to see what shows up where.

In [12]:
words_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
words_df.head(10)

,01,08,09,10,100,101,103rd,10time,10x,10yr,11,112,112th,118,11th,11yr,12,12yr,13,14,14yr,15,15yr,16,16yr,17,18,18th,18yr,19,1cic,1s,1st,1x,1yr,20,20yr,21,22,23,23rd,24,25,25yr,28th,2gether,2mos,2nd,2time,2x,...,witmer,wk,wks,wlkd,wlkng,wndw,wo,woke,woman,women,wood,wooden,woodman,word,work,worker,workplac,would,wouldnt,wound,wout,wrap,wre,wrench,wrestl,wrist,write,wrong,wrote,ws,wth,wtih,wy,xboyfriend,xs,yall,yank,yard,year,yell,yellow,yes,yo,you,younger,your,yourself,yr,yrs,zone
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.243392,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.344879,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.183343,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

## Classify

> "SVM models perform better on sparse data than does trees in general. For example in document classification you may have thousands, even tens of thousands of features and in any given document vector only a small fraction of these features may have a value greater than zero."
>
> https://towardsdatascience.com/support-vector-machine-vs-logistic-regression-94cc2975433f

In [13]:
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC

In [14]:
%%time

X = words_df
y = df.serious

clf = LinearSVC()
clf.fit(X, y)

CPU times: user 7.6 s, sys: 2.88 s, total: 10.5 s
Wall time: 17.7 s


LinearSVC()

Here is how accurate the classifier is! What does that mean????

# Evaluate the classifier

In [15]:
# Remember, this gives you the accuracy
cross_val_score(clf, X, y)

array([0.87242637, 0.87194405, 0.87293884, 0.87064782, 0.86838694])

In [17]:
from sklearn.model_selection import cross_validate

scores = cross_validate(clf, X, y, cv=10,
                        scoring=('accuracy', 'precision', 'recall', 'f1'))

# here are some other types of scores
# https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

scores_df = pd.DataFrame(scores)

display(scores_df.round(2))

pd.DataFrame(scores)[
    ['fit_time', 'score_time', 'test_accuracy','test_precision','test_recall','test_f1']]\
    .mean().round(2)

,fit_time,score_time,test_accuracy,test_precision,test_recall,test_f1
0,13.57,0.35,0.87,0.78,0.67,0.72
1,13.83,0.24,0.87,0.77,0.67,0.72
2,14.98,0.69,0.87,0.77,0.67,0.72
3,14.37,0.42,0.87,0.77,0.66,0.71
4,14.51,0.75,0.87,0.78,0.67,0.72
5,14.79,0.25,0.87,0.77,0.67,0.72
6,14.15,0.24,0.87,0.77,0.67,0.72
7,14.21,0.26,0.87,0.78,0.65,0.71
8,14.08,0.29,0.87,0.76,0.66,0.71
9,14.34,0.60,0.87,0.77,0.66,0.71


fit_time          14.28
score_time         0.41
test_accuracy      0.87
test_precision     0.77
test_recall        0.66
test_f1            0.71
dtype: float64

## Peeking inside the classifier
### Which words are most important?

Before we see how our classifier performs, let's take a look at which words point towards a report being either simple or aggravated assault. In this case, we're going to look at the top and bottom 20 terms.

In [18]:
import eli5

eli5.show_weights(clf, vec=vectorizer, top=(20, 20))

ModuleNotFoundError: No module named 'eli5'

If we jump through a couple hoops, we can also have a nice colored graphic!

In [ ]:
# Build a dataframe of what we have above
weights = eli5.explain_weights_df(clf, vec=vectorizer, top=(20, 20))

# Pick colors based on being above or below zero
colors = weights.weight.apply(lambda weight: 'lightblue' if weight > 0 else 'tan')

# Sort it and plot it
weights.sort_values(
    by='weight',
    ascending=True
).plot(
    y='weight',
    x='feature',
    kind='barh',
    figsize=(7,8),
    color=colors,
    legend=False
)


Outputting coefficients to a CSV file

In [ ]:
coefs = pd.DataFrame(clf.coef_, columns = clf.feature_names_in_ ).T.rename(columns = {0:'coef_'})
coefs.to_csv('coefs1.csv')
coefs.sort_values(by='coef_')

# Discussion

Did the machine learn the same rules you did? 

* Does it sound reasonable which terms imply aggravated vs simple assault?
* Which ones are misspellings? Does that worry you?
* Are there any terms in there you don't quite get?

There's another notebook - "Inspecting misclassifications" - that goes a bit deeper into these terms and what they may (or may not) mean.

https://investigate.ai/latimes-crime-classification/inspecting-classifications/
